## Convolutional Neural Nets for FKD


In [102]:
# initiate packages for NN - Theano etc.
%matplotlib inline

import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import time

import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
device = theano.config.device # We're using CPUs (for now)
floatConf = theano.config.floatX # Should be 64 bit for CPUs

np.random.seed(0)

In [103]:
# load data

import sys, os

pathArr = os.getcwd().split("/")
scriptPath = '/'.join(map(str, pathArr[:len(pathArr)-1]))
sys.path.append(scriptPath)

# import submit module from our tools subfolder
from tools import submit, getdata
# load data

_loaded = getdata.load_data(0, test=True, nonas=True)

FEATURES = _loaded['features']
print 'Number of features:', len(FEATURES)

# load and scale pixel values to [0, 1]
train_data = np.vstack(_loaded['training']['data']) / 255.
if (floatConf == 'float32'):
    train_data = train_data.astype(np.float32)
else:
    train_data = train_data.astype(np.float64)

# load and scale target coordinates to [-1, 1]
train_labels = _loaded['training']['labels']
train_labels = (train_labels - 48) / 48
if (floatConf == 'float32'):
    train_labels = train_labels.astype(np.float32)
else:
    train_labels = train_labels.astype(np.float64)
    
print 'Training dataset size: ', train_data.shape

test_data = _loaded['test']['data']
print 'Test dataset size: ', test_data.shape

Number of features: 30
Training dataset size:  (2140, 9216)
Test dataset size:  (1783,)


In [111]:
# print train_data.shape, train_data.min(), train_data.max()
# print train_labels.shape, train_labels.min(), train_labels.max()
from lasagne import layers
from lasagne.updates import nesterov_momentum
#from nolearn.lasagne import NeuralNet
import nolearn.lasagne

In [113]:
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet

net1 = NeuralNet(
    layers=[
        ('input', layers.InputLayer),
        ('hidden', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    # layer parameters:
    input_shape=(None, 9216),  # 96x96 input pixels per batch
    hidden_num_units=100,  # number of units in hidden layer
    output_nonlinearity=None,  # output layer uses identity function
    output_num_units=30,  # 30 target values

    # optimization method:
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.9,

    regression=True,  # flag to indicate we're dealing with regression problem
    max_epochs=400,  # we want to train this many epochs
    verbose=1,
    )

net1.fit(train_data, train_labels)

  DenseLayer        	(None, 30)          	produces      30 outputs
  DenseLayer        	(None, 100)         	produces     100 outputs
  InputLayer        	(None, 9216)        	produces    9216 outputs


/usr/local/lib/python2.7/site-packages/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


TypeError: cost must be a scalar.